_By_ **Sagnik Modak** _and_ **Sujata Chaudhury**

# Introduction
There are several steps involved in the conversion of raw text-based data into structured data for mindmap generation. The identification and classification of text based on subject is necessary for finding what group of tags and attributes the different entities and relations within the text belong to so that there is minimal error in tagging them. For example, in the context of Computer Science, the statement _"the root is present at the top of the tree"_ is true, while in Biology, the same is not necessarily be true. The classification of text based on subject is therefore, the first step in our procedure.

# Text Preprocessing
The raw text collected from various sources is processed into sentences, each of which is further classified into list of words. We can achieve this by using regular expressions to take care of the common delimiters separating words. However, we have to take note that just including spaces is not enough. We also have to include quotations, question marks, colons, semi-colons, tabs, newlines, etc.

In [13]:
import re

raw_text = """According to Peter Denning, the fundamental question underlying
computer science is, "What can be (efficiently) automated?" Theory of computation
is focused on answering fundamental questions about what can be computed and what amount
of resources are required to perform those computations. In an effort to answer the first
question, computability theory examines which computational problems are solvable on
various theoretical models of computation. The second question is addressed by computational
complexity theory, which studies the time and space costs associated with different approaches
to solving a multitude of computational problems.""" # source: https://en.wikipedia.org/wiki/Computer_science#Theory_of_computation

words = re.compile(r'[\s,:"\'\(\).;?!]+').split(raw_text)

print(words)

['According', 'to', 'Peter', 'Denning', 'the', 'fundamental', 'question', 'underlying', 'computer', 'science', 'is', 'What', 'can', 'be', 'efficiently', 'automated', 'Theory', 'of', 'computation', 'is', 'focused', 'on', 'answering', 'fundamental', 'questions', 'about', 'what', 'can', 'be', 'computed', 'and', 'what', 'amount', 'of', 'resources', 'are', 'required', 'to', 'perform', 'those', 'computations', 'In', 'an', 'effort', 'to', 'answer', 'the', 'first', 'question', 'computability', 'theory', 'examines', 'which', 'computational', 'problems', 'are', 'solvable', 'on', 'various', 'theoretical', 'models', 'of', 'computation', 'The', 'second', 'question', 'is', 'addressed', 'by', 'computational', 'complexity', 'theory', 'which', 'studies', 'the', 'time', 'and', 'space', 'costs', 'associated', 'with', 'different', 'approaches', 'to', 'solving', 'a', 'multitude', 'of', 'computational', 'problems', '']


Since the manual entry of all these delimiters is tedious and quite frankly, unnecessary, we can use a library that takes care of these for us. Here, we will use the nltk library to "tokenize" the input text into a list of words.

In [14]:
from nltk.tokenize import word_tokenize

words = word_tokenize(raw_text)
words = [ w.lower() for w in words if w.isalnum() ]

print(words)

['according', 'to', 'peter', 'denning', 'the', 'fundamental', 'question', 'underlying', 'computer', 'science', 'is', 'what', 'can', 'be', 'efficiently', 'automated', 'theory', 'of', 'computation', 'is', 'focused', 'on', 'answering', 'fundamental', 'questions', 'about', 'what', 'can', 'be', 'computed', 'and', 'what', 'amount', 'of', 'resources', 'are', 'required', 'to', 'perform', 'those', 'computations', 'in', 'an', 'effort', 'to', 'answer', 'the', 'first', 'question', 'computability', 'theory', 'examines', 'which', 'computational', 'problems', 'are', 'solvable', 'on', 'various', 'theoretical', 'models', 'of', 'computation', 'the', 'second', 'question', 'is', 'addressed', 'by', 'computational', 'complexity', 'theory', 'which', 'studies', 'the', 'time', 'and', 'space', 'costs', 'associated', 'with', 'different', 'approaches', 'to', 'solving', 'a', 'multitude', 'of', 'computational', 'problems']


Now we can filter the stopwords from the list using the nltk english corpus. We need to download this and some additional files if we have not already done so any time in the past. To do this, we just execute the following lines.

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAGNIK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SAGNIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Now we compare each word in the list of words we created in the previous step to check if it is present in the list of english stopwords and if present, remove it from our list. Also, it is beneficial for us to convert our words to lower case for uniformity.

In [16]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

filtered_words = [ w for w in words if not w in stop_words and not w is '' ]
print("Filtered words: ", filtered_words)

rejected_words = [ w for w in words if w not in filtered_words ]
print("Rejected words: ", rejected_words)

Filtered words:  ['according', 'peter', 'denning', 'fundamental', 'question', 'underlying', 'computer', 'science', 'efficiently', 'automated', 'theory', 'computation', 'focused', 'answering', 'fundamental', 'questions', 'computed', 'amount', 'resources', 'required', 'perform', 'computations', 'effort', 'answer', 'first', 'question', 'computability', 'theory', 'examines', 'computational', 'problems', 'solvable', 'various', 'theoretical', 'models', 'computation', 'second', 'question', 'addressed', 'computational', 'complexity', 'theory', 'studies', 'time', 'space', 'costs', 'associated', 'different', 'approaches', 'solving', 'multitude', 'computational', 'problems']
Rejected words:  ['to', 'the', 'is', 'what', 'can', 'be', 'of', 'is', 'on', 'about', 'what', 'can', 'be', 'and', 'what', 'of', 'are', 'to', 'those', 'in', 'an', 'to', 'the', 'which', 'are', 'on', 'of', 'the', 'is', 'by', 'which', 'the', 'and', 'with', 'to', 'a', 'of']


# Subject Classification
Let us now first fix the subjects we will be investigating our text for. Here, we chose a set of 4 subjects from which to pick. Depending upon the specific needs, there may be additional steps involved in inclusion of text related to unknown subjects. One method to deal with that might be to limit the results to a certain confidence threshold below which, the program will report 'unknown' or 'other' instead of the actual result.

In [17]:
subjects = [ 'computer', 'arts', 'economics', 'architecture' ]

Now we need a model for comparing our words with the subject. We can either create our own model using any Machine Learning library such as sklearn, tf, etc. Here, we will use a pre-trained model from Google News using the Gensim api. We then find out the score for each subject as the median similarity of all the words in our list against the subject.

In [20]:
import numpy as np
import gensim.downloader as api

model = api.load('word2vec-google-news-300')

filtered_available_words = []
filtered_unavailable_words = []

for word in filtered_words:
    if not word in model.vocab:
        filtered_unavailable_words.append(word)
    else:
        filtered_available_words.append(word)

similarities = []

for i, subject in enumerate(subjects):
    similarities.append([])
    for word in filtered_available_words:
        value = model.similarity(subject, word)
        similarities[i].append(value)

# We adopt a median based measurement of score
similarities = np.array(similarities)
scores = np.median(similarities, axis=1)
print("Unknown words found: %d"%(len(filtered_unavailable_words)))
print("Scores: ", scores)

Unknown words found: 0
Scores:  [0.12262617 0.04587922 0.13724777 0.09913225]


As we can see, the score for the subject "computer" is not the most. Although, it is still very close to the highest scored subject amongst all subjects in the list. We can see that even though the subject is clearly "computer", the prediction goes awry and the scores for all the subjects are very close. We do not want that. We want a clear distinction between the different subjects. There might be a few different reasons for that -
1. The training dataset might not be ideal or might be missing crucial words that can change the output significantly. We need to change our dataset to a suitable one.
2. The subjects belong to a similar group of words (here for example, they are all different disciplines of "science") and therefore, bear overlapping similarities with words in our list. We can apply novel techniques.
3. Our scoring procedure might be flawed. We need to find an optimal scoring method.

Let us tackle the problems one by one.

## Problem 1: Traning Dataset
We already found that for our specific example, we do not have any unknown words in our list. This means that we can safely assume that the results are not impacted in any manner by the absence of critical words in the dataset. However, this might not be true in all scenarios. Let us take this a step further. Suppose some word in this list of unknown words was significantly consequential to any subject. If we are studying a large enough corpus of text and the subjects have enough dissimilarity between them, the absence of a word in the dataset will not impact the entire topic extraction process as a whole very drastically. We have to set proper guidelines as to when the absence of critical words is impactful enough so that we need to change our model. One of the ways this can be achieved is by setting a threshold on the maximum contribution of these words in the text. It is to be noted that this method is partial to long words with no semantic similarities to the text and is by no way meant to be taken as for actual use, but is presented here due to it's simplicity and easy implementation.

In [16]:
threshold = 5.

unknowns_contrib = 0

for word in filtered_words:
    if not word in model:
        unknowns_contrib += len(word)

total_size = len(''.join(filtered_words))

percentage = 100 * unknowns_contrib / total_size # Calculates between 0 and 100

print('Percentage: %f%%'%(percentage))

if percentage > threshold:
    print('The model is not suitable for this data')
else:
    print('The model is suitable for this data')

Percentage: 0.000000%
The model is suitable for this data


Another problem related to training dataset might be the actual word vectors themselves. We can get a comparison by running the same set of rules for a different dataset based on an entirely different source. In this example, we will consider the wikipedia dataset and compare the results with

In [21]:
import numpy as np
import gensim.downloader as api

model = api.load('glove-wiki-gigaword-50')

filtered_available_words = []
filtered_unavailable_words = []

for word in filtered_words:
    if not word in model.vocab:
        filtered_unavailable_words.append(word)
    else:
        filtered_available_words.append(word)

similarities = []

for i, subject in enumerate(subjects):
    similarities.append([])
    for word in filtered_available_words:
        value = model.similarity(subject, word)
        similarities[i].append(value)

# We adopt a median based measurement of score
similarities = np.array(similarities)
scores = np.median(similarities, axis=1)
print("Unknown words found: %d"%(len(filtered_unavailable_words)))
print("Scores: ", scores)

[==================================================] 100.0% 66.0/66.0MB downloaded
Unknown words found: 0
Scores:  [0.48503646 0.30647114 0.39660767 0.34896588]


We observe that the results change significantly upon selecting a different dataset. This is somewhat expected as the raw text was from Wikipedia page on Computer Science. We also note that the score for "computer" is now more than that for "economics" which is correct. We will still follow through the next problems and see if (and how) we can improve on this result.

## Problem 2: Subject similarities
Let us now focus on the second part of our problem. If the subjects in our list are similar to each other and belong to the same broad category, there will be some overlapping between them.